In [1]:
import sympy as smp

In [37]:
v_x, v_y, v_z, v = smp.symbols('v_x v_y v_z v')
omega_w, i_w = smp.symbols('omega_w i_w')

In [62]:
# Rotation matrix around the z-axis by the omega_wind angle
R_z = smp.Matrix([[smp.cos(omega_w), -smp.sin(omega_w), 0],
                    [smp.sin(omega_w), smp.cos(omega_w), 0],
                    [0, 0, 1]])

# # Rotation matrix around the y-axis by the wind inclination angle
# R_y = smp.Matrix([[smp.cos(i_w), 0, smp.sin(i_w)],
#                     [0, 1, 0],
#                     [-smp.sin(i_w), 0, smp.cos(i_w)]])


In [63]:
# Initial velocity vector (v_z=0 as in XY plane)
v = smp.Matrix([v_x, v_y, 0]) 

In [64]:
# Rotating vector around z-axis by omega_wind
v_p = R_z * v
v_p

Matrix([
[v_x*cos(omega_w) - v_y*sin(omega_w)],
[v_x*sin(omega_w) + v_y*cos(omega_w)],
[                                  0]])

In [65]:
# Angles to rotate by (in radians)
omega = -23 * smp.pi / 180
i = 22 * smp.pi / 180

# Velocity vector for WN star and WC star at orbital phase of 0.7
v_WN = {v_x: 635666, v_y: -162950, v_z: 0}
s_WN = {v_x: -5.44133e+14, v_y: -3.52402e+14, v_z: 0}

v_WC = {v_x: -847554, v_y: 217267, v_z: 0}
s_WC = {v_x: 7.25511e+14, v_y: 4.69869e+14, v_z: 0}

In [53]:
# Rotating j vector around z-axis by omega_wind
j = smp.Matrix([0, 1, 0])
j_p = R_z * j
j_p = j_p.subs(omega_w, omega)
j_p

Matrix([
[sin(23*pi/180)],
[cos(23*pi/180)],
[             0]])

In [54]:
u_x, u_y, u_z = smp.symbols('u_x u_y u_z')
u = smp.Matrix([u_x, u_y, u_z])

# Rodrigues' rotation formula for rotation around an arbitrary axis
W = smp.Matrix([[0, -u_z, u_y],
                [u_z, 0, -u_x],
                [-u_y, u_x, 0]])
R_u = smp.eye(3) + smp.sin(i_w) * W + (1 - smp.cos(i_w)) * W**2
R_u

Matrix([
[(1 - cos(i_w))*(-u_y**2 - u_z**2) + 1, u_x*u_y*(1 - cos(i_w)) - u_z*sin(i_w),  u_x*u_z*(1 - cos(i_w)) + u_y*sin(i_w)],
[u_x*u_y*(1 - cos(i_w)) + u_z*sin(i_w), (1 - cos(i_w))*(-u_x**2 - u_z**2) + 1, -u_x*sin(i_w) + u_y*u_z*(1 - cos(i_w))],
[u_x*u_z*(1 - cos(i_w)) - u_y*sin(i_w), u_x*sin(i_w) + u_y*u_z*(1 - cos(i_w)),  (1 - cos(i_w))*(-u_x**2 - u_y**2) + 1]])

In [66]:
# Rotate v_prime around the axis j_prime by the inclination angle
v_pp = R_u * v_p
v_pp

Matrix([
[(v_x*sin(omega_w) + v_y*cos(omega_w))*(u_x*u_y*(1 - cos(i_w)) - u_z*sin(i_w)) + (v_x*cos(omega_w) - v_y*sin(omega_w))*((1 - cos(i_w))*(-u_y**2 - u_z**2) + 1)],
[(v_x*sin(omega_w) + v_y*cos(omega_w))*((1 - cos(i_w))*(-u_x**2 - u_z**2) + 1) + (v_x*cos(omega_w) - v_y*sin(omega_w))*(u_x*u_y*(1 - cos(i_w)) + u_z*sin(i_w))],
[(u_x*sin(i_w) + u_y*u_z*(1 - cos(i_w)))*(v_x*sin(omega_w) + v_y*cos(omega_w)) + (v_x*cos(omega_w) - v_y*sin(omega_w))*(u_x*u_z*(1 - cos(i_w)) - u_y*sin(i_w))]])

In [68]:
# Subbing in values
v_pp = v_pp.subs({omega_w: omega, i_w: i, u_x: j_p[0], u_y: j_p[1], u_z: j_p[2]})
v_pp

Matrix([
[(1 - cos(11*pi/90))*(-v_x*sin(23*pi/180) + v_y*cos(23*pi/180))*sin(23*pi/180)*cos(23*pi/180) + (v_x*cos(23*pi/180) + v_y*sin(23*pi/180))*(-(1 - cos(11*pi/90))*cos(23*pi/180)**2 + 1)],
[(-v_x*sin(23*pi/180) + v_y*cos(23*pi/180))*(-(1 - cos(11*pi/90))*sin(23*pi/180)**2 + 1) + (1 - cos(11*pi/90))*(v_x*cos(23*pi/180) + v_y*sin(23*pi/180))*sin(23*pi/180)*cos(23*pi/180)],
[                                     (-v_x*sin(23*pi/180) + v_y*cos(23*pi/180))*sin(11*pi/90)*sin(23*pi/180) - (v_x*cos(23*pi/180) + v_y*sin(23*pi/180))*sin(11*pi/90)*cos(23*pi/180)]])

In [69]:
# Subbing in values for the velocities, positions of the stars
# WC Star 
v_rotated1 = v_pp.subs(v_WC)
s_rotated1 = v_pp.subs(s_WC)

# WN Star
v_rotated2 = v_pp.subs(v_WN)
s_rotated2 = v_pp.subs(s_WN)

In [70]:
# WC star velocities
smp.N(v_rotated1, 7)

Matrix([
[-638475.1],
[ 507046.8],
[ 317499.3]])

In [71]:
# WC Star positions
smp.N(s_rotated1, 7)

Matrix([
[ 8.027996e+14],
[ 1.696789e+14],
[-2.717812e+14]])

In [72]:
# WN star velocities
smp.N(v_rotated2, 7)

Matrix([
[ 478856.8],
[-380285.1],
[-238124.7]])

In [73]:
# WN Star positions
smp.N(s_rotated2, 7)

Matrix([
[-6.020996e+14],
[-1.272595e+14],
[ 2.038358e+14]])